In [ ]:
import numpy as np
import pickle

#from train_model import train_step, test_step
from models.decisiontree import DecisionTree, RandForest
from utils.load_data import get_data
from utils.make_dict import train_bow, get_bow

In [ ]:
args ={'dataset': 'cifar10',
       'dataroot': './data',
       'model': 'custom_SVM',
       'kernel': 'gaussian',
       'validation': 0.1,
       'dict_size': 300,
       'load_cluster': False
       }

In [ ]:
trainX, trainy = get_data(dataset=args['dataset'], train=True, dataroot=args['dataroot'])
if args['dataset'] == 'cifar10':
    trainX = trainX.reshape((-1, 32, 32, 3), order='F')

if args['load_cluster']:
    with open("./cluster.dump", "rb") as f:
        cluster = pickle.load(f)
else:
    cluster = train_bow(trainX, num_dict=args['dict_size'], num_select=10000)
    with open("./cluster.dump", "wb") as f:
        pickle.dump(cluster, f)

trainFeature = get_bow(trainX, cluster, num_dict=args['dict_size'])

In [ ]:
model = RandForest(forest=10, bag_size=5000, depth=100)
#model = DecisionTree(depth=150)

In [ ]:
model.fit(trainFeature, trainy)

In [ ]:
prediction, pred = model.predict(trainFeature)

In [ ]:
trainFeature.shape

In [ ]:
boolean = (pred == trainy)
print(np.sum(boolean) / trainy.shape[0] * 100)

In [ ]:
with open("./cluster.dump", "rb") as f:
    cluster = pickle.load(f)
testX, testy = get_data(dataset=args['dataset'], train=False, dataroot=args['dataroot'])
if args['dataset'] == 'cifar10':
    testX = testX.reshape((-1, 32, 32, 3), order='F')
testFeature = get_bow(testX, cluster, num_dict=args['dict_size'])

In [ ]:
pred = model.predict(testFeature)

In [ ]:
boolean = (pred[1] == testy)

In [ ]:
print(np.sum(boolean) / testy.shape[0] * 100)